In [ ]:
import tensorflow as tf
import numpy as py
import load_data as ld
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
#  计算F1的值，可以选择softmax或sigmoid函数的输出用于计算
def single_f1(y_pred, y_label, activation='softmax'):
    y_true = [[]]
    if activation == 'softmax':  
        for item in y_label.numpy():
            y_true[0].append(item)
        y_true = tf.constant(y_true)
        y_hat = []
        for item in y_pred:
            print(item)
            if item[0] > item[1]:
                y_hat.append([0.])
            else:
                y_hat.append([1.])
        y_hat = tf.constant(y_hat)
    if activation == 'sigmoid':
        for item in y_label.numpy():
            y_true[0].append(item)
        y_true = tf.constant(y_true)
        y_hat = []
        for item in y_pred:
            if item.numpy() < 0.5:
                y_hat.append([0.])
            else:
                y_hat.append([1.])
        y_hat = tf.constant(y_hat)
        
    epsilon = 1e-7
  
    tp = tf.cast(tf.matmul(y_true,y_hat), 'float')
    #tn = tf.sum(tf.cast((1-y_hat)*(1-y_true), 'float'), axis=0)
    fp = tf.cast(tf.matmul(1-y_true,y_hat), 'float')
    fn = tf.cast(tf.matmul(y_true,1-y_hat), 'float')
    print('tp: ', tp[0][0].numpy(),'fp: ',  fp[0][0].numpy(),'fn: ',  fn[0][0].numpy())
    p = tp/(tp+fp+epsilon)#epsilon的意义在于防止分母为0，否则当分母为0时python会报错
    r = tp/(tp+fn+epsilon)
    
    f1 = 2*p*r/(p+r+epsilon)
    
    result = [f1[0][0].numpy(), p[0][0].numpy(), r[0][0].numpy()]
    
    print('F1-score: ',result[0])
    print('precision: ',result[1])
    print('recall: ',result[2])
    print('-----------------------------------------------')
    return result

In [ ]:
#  统计文本长度数据
def count_segment_amount(text):
    text = ld.split_word(text)
    length_count = []
    for item in text:
        length_count.append(len(item))
    mpl.rcParams['font.sans-serif'] = ['SimHei']
    mpl.rcParams['axes.unicode_minus'] = False
    plt.ylabel("文本数量")
    plt.xlabel("文本长度")
    plt.hist(length_count, bins=40, facecolor="grey", edgecolor="black", alpha=0.7)

In [ ]:
#  统计类别样本数量，内部调用了load_data.transform_to_multi_class()方法，将多标签为多个二类
def count_label_amount(label):
    label = ld.transform_to_multi_class(label)
    type_dic = {'Introductory/Generic': 0, 'Practice not covered': 1,
            'Privacy contact information': 2, 'User Access, Edit and Deletion': 3,
            'Data Security': 4, 'International and Specific Audiences': 5,
            'Do Not Track': 6, 'User Choice/Control': 7,
            'Data Retention': 8, 'Policy Change': 9,
            'First Party Collection/Use': 10, 'Third Party Sharing/Collection': 11}
    type_list = []
    for item in type_dic:
        type_list.append(item)
    count_type = []
    for item in label:
        count = 0
        for type in item:
            if type.numpy() == 1:
                count = count+1
        count_type.append(count)
    mpl.rcParams['font.sans-serif'] = ['SimHei']
    mpl.rcParams['axes.unicode_minus'] = False
    x = range(len(count_type))
    rects = plt.bar(x=x, height=count_type, width=0.4, alpha=0.8, color='grey', label="类别")
    plt.ylim(0, 1600)     # y轴取值范围
    plt.ylabel("类别数量")
    plt.xticks([index for index in x], type_list,rotation=90)
    plt.xlabel("类别")
    plt.legend()    
    for rect in rects:
        height = rect.get_height()
        plt.text(rect.get_x() + rect.get_width() / 2, height+1, str(height), ha="center", va="bottom")
    plt.show()